In [1]:
import glob
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img, img_to_array

In [4]:
from tensorflow.keras.models import Sequential

In [5]:
train_img_names = glob.glob('./data/train/cat/*')
print(type(train_img_names))
print(len(train_img_names))

<class 'list'>
9499


In [6]:
demo_img_0 = train_img_names[0]
demo_img_0

'./data/train/cat\\cat.10000.jpg'

In [7]:
IMG_DIM = (150, 150)

In [8]:
train_img_raw = img_to_array(load_img(demo_img_0, target_size = IMG_DIM))
print(type(train_img_raw))
print(train_img_raw.shape)

<class 'numpy.ndarray'>
(150, 150, 3)


In [9]:
train_img_raw.max()

255.0

In [10]:
train_img_raw.min()

0.0

In [11]:
rescale_coef = np.divide(1, train_img_raw.max())
rescale_coef

0.00392156862745098

In [12]:
train_data_gen = ImageDataGenerator(
                                    rescale = rescale_coef,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True,
                                    vertical_flip = True
                                   )

In [13]:
test_data_gen = ImageDataGenerator(rescale = rescale_coef)

In [14]:
train_generator = train_data_gen.flow_from_directory(
                                                    './data/train',
                                                    target_size = IMG_DIM,
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )

Found 18998 images belonging to 2 classes.


In [15]:
test_generator = train_data_gen.flow_from_directory(
                                                    './data/test',
                                                    target_size = IMG_DIM,
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )

Found 6002 images belonging to 2 classes.


In [16]:
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPooling2D, Dropout

In [17]:
model = Sequential()

model.add(
        Conv2D(
                filters = 32,
                kernel_size = (3, 3),
                strides = 1,
                padding = 'same',
                input_shape = train_img_raw.shape,
                activation = 'relu'
            )
        )

model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(
        Conv2D(
                filters = 32,
                kernel_size = (3, 3),
                strides = 1,
                padding = 'same',
                activation = 'relu'
            )
        )

model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(
        Conv2D(
                filters = 64,
                kernel_size = (3, 3),
                strides = 1,
                padding = 'same',
                activation = 'relu'
            )
        )

model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(units = 64, activation = 'relu'))

model.add(Dropout(0.5))

model.add(Dense(units = 1, activation = 'sigmoid'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20736)             0

In [19]:
model.compile(optimizer = 'adam', loss = 'binary_crossentroy', metrics = ['accuracy'])

In [20]:
model.fit(
        train_generator,
        validation_data = test_generator,
        epochs = 1,
        steps_per_epoch = 9499 // 32
         )

ImportError: Image transformations require SciPy. Install SciPy.